# Visualise results from model trained in clustering_analysis_training.ipynb

Author: Katharine Leney, April 2025

In [1]:
from bertopic import BERTopic # topic modelling
import pandas as pd
import pickle
import os
import json
import sys
sys.path.append("../src")
from utils.parse_model_folder import parse_model_folder
from utils.generate_clean_labels import generate_clean_labels
from utils.generate_opportunity_table import generate_opportunity_table

In [2]:
# ====================================
# Load model and associated data
# ====================================

# Model trained on full dataset
#model_name = f"bertopic_cluster_model_20250425"

# Model trained on 2015--2019, 2022--2024 data
model_name = f"bertopic_cluster_model_from_2015_excl_2020_2021_20250426"

# Load model
topic_model = BERTopic.load(os.path.join("../models/", model_name, "model"))

# Load texts and timestamps
with open(os.path.join("../models/", model_name, "texts.pkl"), "rb") as f:
    texts = pickle.load(f)

with open(os.path.join("../models/", model_name, "timestamps.pkl"), "rb") as f:
    timestamps = pickle.load(f)

model_label = parse_model_folder(model_name)
print(model_label)

# Make sure the outputs folder exists
os.makedirs("../outputs", exist_ok=True)

From 2015 | Excluding 2020, 2021


In [3]:
# Explore top topics
topic_model.get_topic_info().head(10)
#topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,142,-1_fuel_airlines_s_timatic,"[fuel, airlines, s, timatic, airport, jet fuel...","[60 IATA OFFICES Geneva Executive Office 33,..."
1,0,201,0_passenger_year_cargo_growth,"[passenger, year, cargo, growth, load, demand,...",[Africa Russia Brazil Jan Feb Mar Apr May Jun ...
2,1,151,1_saf_emissions_carbon_corsia,"[saf, emissions, carbon, corsia, environmental...",[Environment & Sustainability 28 IATA Annual ...
3,2,147,2_safety_iosa_accidents_accident,"[safety, iosa, accidents, accident, aircraft, ...",[Safety continues to improve The last decade s...
4,3,110,3_tax_regulation_taxation_passenger,"[tax, regulation, taxation, passenger, unruly,...",[The industry is asking governments to adopt s...
5,4,108,4_cargo_data_lithium_ceiv,"[cargo, data, lithium, ceiv, e, freight, goods...",[Special cargo Standardized air cargo processe...
6,5,102,5_airport_charges_infrastructure_airports,"[airport, charges, infrastructure, airports, d...",[Cost-efficiency and lower charges Infrastruct...
7,6,98,6_security_information_smart security_smart,"[security, information, smart security, smart,...",[The UN resolution confirms ICAO’s leadership...
8,7,64,7_settlement_billion_processed_iss,"[settlement, billion, processed, iss, bsp, fin...","[At the end of 2017, CASS was processing 94 ..."
9,8,60,8_ndc_order_travel_airline,"[ndc, order, travel, airline, retailing, airli...",[IATA has drafted a recommended practice for ...


In [4]:
# ------------------------------------------------------------
# Visualize intertopic distances
# ------------------------------------------------------------

# N.B. UMAP (Uniform Manifold Approximation and Projection)
# used for dimensionality reduction. 
fig_2d = topic_model.visualize_topics(custom_labels=True)

# Update background and grid
fig_2d.update_layout(
    plot_bgcolor="white",
    paper_bgcolor="white",
    font_family="Arial",
    title={
        "text": f"Intertopic Distance Map<br><sup>{model_label}</sup>",
        "x": 0.15,
        "font": {"size": 20}
    },
    width=1000,
    height=800,
    xaxis=dict(
        showgrid=False,
        zeroline=True,
        showline=True,
        ticks="outside",
        title=None
    ),
    yaxis=dict(
        showgrid=False,
        zeroline=True,
        showline=True,
        ticks="outside",
        title=None
    ),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="center",
        x=0.5
    )
)
fig_2d.write_html("../outputs/intertopic_distance_map_2D.html")   

fig_2d.show()

In [5]:
# ------------------------------------------------------
# Clean up labels and remove junk topics
# ------------------------------------------------------

# Retrieve the current model labels before overwriting
old_labels = topic_model.topic_labels_

# Generate new labels
clean_labels, junk_topics = generate_clean_labels(topic_model, top_n_words=1)

# Print label changes for review
print("\nGenerated topic labels:")
for topic_id, label in sorted(clean_labels.items()):
    old_label = old_labels.get(topic_id)
    print(f"Topic {topic_id}: {label} (OLD: {old_label})")

# Apply to the model
topic_model.set_topic_labels(clean_labels)


Generated topic labels:
Topic -1: NULL (OLD: -1_fuel_airlines_s_timatic)
Topic 0: Passenger (OLD: 0_passenger_year_cargo_growth)
Topic 1: SAF (OLD: 1_saf_emissions_carbon_corsia)
Topic 2: Safety (OLD: 2_safety_iosa_accidents_accident)
Topic 3: Tax (OLD: 3_tax_regulation_taxation_passenger)
Topic 4: Cargo (OLD: 4_cargo_data_lithium_ceiv)
Topic 5: Airport (OLD: 5_airport_charges_infrastructure_airports)
Topic 6: Security (OLD: 6_security_information_smart security_smart)
Topic 7: Settlement (OLD: 7_settlement_billion_processed_iss)
Topic 8: New Distribution Capability (OLD: 8_ndc_order_travel_airline)
Topic 9: Commercial (OLD: 9_commercial_world_contents_s)
Topic 10: Travel (OLD: 10_travel_id_experience_passengers)
Topic 11: Chairman (OLD: 11_chairman_president_airlines_board)
Topic 12: Airlines (OLD: 12_airlines_airways_latam_china)
Topic 13: Slot (OLD: 13_slot_wsg_slots_regulation)
Topic 14: Diversity (OLD: 14_diversity_inclusion_diversity inclusion_female)
Topic 15: Training (OLD: 15

In [6]:
# ------------------------------------------------------------
# Generate topic time series and visualize trends
# ------------------------------------------------------------

timestamps = pd.to_datetime(timestamps, format="%Y")
topics_over_time = topic_model.topics_over_time(texts, timestamps, topic_model.topics_)

# Save time series data and topic labels for the RAG assistant to use later
topics_over_time.to_csv("../outputs/topics_over_time.csv", index=False)
with open("../outputs/topic_labels.json", "w") as f:
    json.dump(clean_labels, f, indent=2)

# Filter out junk and noise topics from list and count rate
topic_freq = topic_model.get_topic_freq()
valid_topic_freq = topic_freq[
    (~topic_freq["Topic"].isin(junk_topics)) &
    (topic_freq["Topic"] != -1)
]

# Select top 10 most frequent clean topics
top_topics = valid_topic_freq.head(10)["Topic"].tolist()

# Make plot of topic frequency by year
# y-axis = % of all text chunks assigned to this topic in a given year
fig = topic_model.visualize_topics_over_time(topics_over_time, topics=top_topics)

# Apply custom colors and labels
for i, trace in enumerate(fig.data):
    topic_id = top_topics[i]
    #trace.line.color = iata_palette[i % len(iata_palette)]
    trace.name = clean_labels[topic_id]
    hover_text = trace.hovertemplate or ""
    trace.hovertemplate = f"{clean_labels[topic_id]}<extra></extra>"

# Clean up the figure layout
fig.update_layout(
    title=f"Topic Trends Over Time<br><sup>{model_label}</sup>",
    width=1200,
    height=600,
    xaxis_title="Year",
    yaxis_title="Topic Frequency (% of Annual Report)",
    legend_title="Topics",
    font=dict(family="Arial", size=12),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.6,
        xanchor="center",
        x=0.5
    )
)

output_filename = f"../outputs/topic_trends_{model_name}"
fig.write_html(f"{output_filename}.html")
fig.write_image(f"{output_filename}.png")

fig.show()

8it [00:00,  8.26it/s]


In [7]:
# ===============================================
# Generate a table matching trends to business
# opportunities, and assign a priority ranking
# based on trend size and growth
# ===============================================

#topics_over_time = topic_model.topics_over_time(texts, timestamps, topic_model.topics_)
matched_opportunities_df, unmatched_topics = generate_opportunity_table(topic_model, topics_over_time)

print(f"Filtering (if any): {model_label}")
print(f"Model Name: {model_name}\n")
print("=== Matched Business Opportunities ===\n")
display(matched_opportunities_df)

# Save table as CSV
output_filename = f"../outputs/business_opportunities_{model_name}.csv"
matched_opportunities_df.to_csv(output_filename, index=False)

print(f"\nSaved opportunity table to: {output_filename}")

Filtering (if any): From 2015 | Excluding 2020, 2021
Model Name: bertopic_cluster_model_from_2015_excl_2020_2021_20250426

=== Matched Business Opportunities ===



,Trend,Detected Label,Implication,Opportunity,Priority (Recommended),Keyword Fraction (%),Topic Growth
0,Growth,0_passenger_year_cargo_growth,Strong recovery in air travel post-pandemic,Expand network capacity and optimize route pla...,High,15.76,10
1,Emissions,1_saf_emissions_carbon_corsia,Increasing pressure for environmental responsi...,Adopt Sustainable Aviation Fuels (SAF) and car...,High,11.84,16
2,Safety,2_safety_iosa_accidents_accident,Continued industry focus on safety standards,Invest in IOSA certification and safety audits,Medium,11.53,-3
3,Regulation,3_tax_regulation_taxation_passenger,Tighter regulatory environment for passenger r...,Implement compliance monitoring and customer c...,Medium,8.63,-5
4,Cargo,4_cargo_data_lithium_ceiv,Continued strength in air cargo and freight de...,Expand cargo services and invest in logistics ...,Medium,8.47,-7
5,Airport,5_airport_charges_infrastructure_airports,Growing airport-related fees and costs,Negotiate airport charges and improve operatio...,Medium,8.00,-1
6,Security,6_security_information_smart security_smart,Passenger data security and travel security co...,Enhance digital identity management and passen...,Medium,7.69,-1
7,Settlement,7_settlement_billion_processed_iss,"Changes in financial settlement systems (BSP, ...",Optimize settlement systems and implement fast...,Medium,5.02,-2
8,ID,10_travel_id_experience_passengers,Move towards digital identity and seamless travel,Implement biometric and digital ID solutions,Medium,3.22,7
9,Regulation,13_slot_wsg_slots_regulation,Tighter regulatory environment for passenger r...,Implement compliance monitoring and customer c...,Medium,2.27,2



Saved opportunity table to: ../outputs/business_opportunities_bertopic_cluster_model_from_2015_excl_2020_2021_20250426.csv
